In [73]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [74]:

url_g="https://baniola.tn/voitures"

nbre_page=39
# nbre_page=1
n=0

marques = []
modeles = []
years = []
puissances_fiscales = []
prices = []
kilometrages = []
boites = []
fuels = []  
cylindres = []
nbre_portes = []
prices=[]
negotiables = []
total_df = pd.DataFrame()

In [ ]:
for n in range(nbre_page):
    url = url_g + "?page=" + str(n * 50)
    code_page = requests.get(url)
    soup = BeautifulSoup(code_page.text, 'html.parser')

    print('The URL now is:', url)
    
    # Extract car links for detailed information
    links_container = soup.findAll('div', class_='card-body')
    links = [
        item.find('a', class_='card-title text-capitalize', href=True)['href']
        for item in links_container if item.find('a', class_='card-title text-capitalize', href=True)
    ]

    # Extract features and prices for each car
    for link in links:
        print('-------------------')
        page_voiture = requests.get(link)
        soup_voiture = BeautifulSoup(page_voiture.text, 'html.parser')

        # Extract price information
        prices_container = soup_voiture.findAll('div', class_='price')
        for price in prices_container:
            price_text = price.text.strip()
            # Extract the numeric value from price text
            price_number = re.search(r'\d+', price_text.replace('TND', '').replace(' ', ''))
            prices.append(price_number.group() if price_number else np.nan)

        # Extract car features
        div_features = soup_voiture.find('div', class_='features')
        details = div_features.findAll('div', class_='detail') if div_features else []

        # Extract labels and values into a dictionary
        features = {}
        for detail in details:
            label = detail.find('div', class_='label').text.strip().replace(':', '').strip().lower()
            value = detail.find('div', class_='value').text.strip()
            features[label] = value

        # Print the extracted dictionary
        print(features)

        # Append extracted data to respective lists
        marques.append(features.get('marque', np.nan))
        modeles.append(features.get('modèle', np.nan))
        years.append(features.get('année', np.nan))
        puissances_fiscales.append(features.get('puissance fiscale', np.nan))
        kilometrages.append(features.get('kilométrage', np.nan))
        boites.append(features.get('boîte de vitesses', np.nan))
        fuels.append(features.get('carburant', np.nan))
        cylindres.append(features.get('cylindrée', np.nan))
        nbre_portes.append(features.get('nombre de portes', '4'))

    # Ensure consistent lengths before creating the DataFrame
    if len(prices) == len(marques) == len(modeles) == len(kilometrages) == len(years) == len(boites) == len(puissances_fiscales) == len(fuels) == len(cylindres) == len(nbre_portes):
        data = {
            'Marque': marques,
            'Modèle': modeles,
            'year': years,
            'road': kilometrages,
            'boite': boites,
            'horsepower': puissances_fiscales,
            'fuel': fuels,
            'nbre_portes': nbre_portes,
            'cylindrée': cylindres,
            'Prix': prices
        }
        df = pd.DataFrame(data)

    # Append to total DataFrame
    total_df = pd.concat([total_df, df], ignore_index=True)

# Save to CSV
total_df.to_csv('../data/baniola.csv', index=False)

print("Data saved to '../data/baniola.csv'")


The URL now is: https://baniola.tn/voitures?page=0
-------------------
{'marque': 'Hyundai', 'modèle': 'I10', 'année': '2015', 'puissance fiscale': '5 CV', 'kilométrage': '180 000 KM', 'boîte de vitesses': 'Manuelle', 'carburant': 'Essence', 'nombre de portes': '5'}
-------------------
{'marque': 'Suzuki', 'modèle': 'Autres', 'année': '2020', 'puissance fiscale': '5 CV', 'kilométrage': '90 000 KM', 'boîte de vitesses': 'Manuelle', 'carburant': 'Essence', 'cylindree': '3 L', 'nombre de portes': '5'}
-------------------
{'marque': 'Suzuki', 'modèle': 'Autres', 'année': '2020', 'puissance fiscale': '5 CV', 'kilométrage': '88 000 KM', 'boîte de vitesses': 'Manuelle', 'carburant': 'Essence', 'cylindree': '3 L', 'nombre de portes': '5'}
-------------------
{'marque': 'Mahindra', 'modèle': 'KUV 100', 'année': '2022', 'puissance fiscale': '5 CV', 'kilométrage': '10 300 KM', 'boîte de vitesses': 'Manuelle', 'carburant': 'Diesel', 'cylindree': '3 L'}
-------------------
{'marque': 'Peugeot', 'mo

In [76]:
print(df)
print(total_df.duplicated().sum())
for item in total_df.duplicated():
    if item == True:
        print(item)


        Marque         Modèle  year        road        boite horsepower  \
0      Hyundai            I10  2015  180 000 KM     Manuelle       5 CV   
1       Suzuki         Autres  2020   90 000 KM     Manuelle       5 CV   
2       Suzuki         Autres  2020   88 000 KM     Manuelle       5 CV   
3     Mahindra        KUV 100  2022   10 300 KM     Manuelle       5 CV   
4      Peugeot            308  2015  180 000 KM     Manuelle       5 CV   
...        ...            ...   ...         ...          ...        ...   
1944   Peugeot            508  2012  280 000 KM     Manuelle       9 CV   
1945      Audi             A4  2018  114 000 KM  Automatique       8 CV   
1946   Renault         Koleos  2014   96 000 KM     Manuelle       8 CV   
1947  Mahindra          AUTRE  2022   10 500 KM     Manuelle       5 CV   
1948      Fiat  PALIO WEEKEND  2001  300 000 KM     Manuelle       4 CV   

         fuel nbre_portes cylindrée    Prix  
0     Essence           5       NaN   28000  
1     E